# Analyse de données de films

Nous proposons de travailler sur des données décrivant des films. Les possibilités sont larges et vous êtes évalués sur vos propositions et votre méthodologie plus que sur vos résultats.

Les données de départ sont disponibles sur:
https://grouplens.org/datasets/movielens/
au format CSV. 

Nous nous intéresserons en particulier au jeu de données: **MovieLens 20M Dataset**. Dans ce jeu de données, vous disposez entre autre de:
* Idendifiant du film dans IMdb et TMdb (ça sera important ensuite)
* Catégorie(s) du film
* Titre du film
* Notes données par les internautes aux films

Afin de rendre le projet plus intéressant, nous ajoutons des données sur les acteurs et producteurs associés aux films (récupéré sur TMdb). Ces données sont disponibles sur les liens suivants:

http://webia.lip6.fr/~guigue/film_v2.pkl <br>
http://webia.lip6.fr/~guigue/act_v2.pkl <br>
http://webia.lip6.fr/~guigue/crew_v2.pkl

Ces fichiers contiennent respectivement : une nouvelle description des films (dont l'identifiant TMdb et la note moyenne donnée par les internautes, la date de sortie,...), une description des acteurs de chaque film et une description des équipes (scénariste, producteur, metteur en scène) pour chaque film.

Ces données sont des listes de taille 26908, chaque élément de la liste correspondant à un dictionnaire dont vous étudierez les clés pour récupérer les informations utiles.

**ATTENTION** Les contraintes de récupération d'informations en ligne font que la base MovieLens compte 27278 films mais les fichiers ci-dessus n'en comptent que 26908. Le plus simple est probablement d'éliminer les films de MovieLens qui ne sont pas dans cette seconde base.

## Consignes générales pour l'analyse des données

Vous devez proposer plusieurs analyses des données, qui devront à minima utiliser les
 techniques suivantes:
 
1. Mettre en forme les données pour identifier les acteurs et les catégories, les indexer
1. Traiter au moins un problème de régression supervisé (par exemple la prédiction de la note moyenne donnée à un film par les internautes).
1. Traiter au moins un problème de classification supervisé (par exemple la prédiction de la catégorie d'un film)
1. Utiliser les données catégorielles (catégories, acteurs,...) de manière discrète ET de manière coninue (*dummy coding*) dans des approches différentes
1. Proposer au moins une approche de catégorisation non supervisée (pour regrouper les acteurs par exemple)
1. Mener une campagne d'expérience permettant de comparer les performances sur un problème en fonction des valeurs d'un paramètre (et donc, in fine, trouver la meilleure valeur du paramètre)
1. Proposer quelques illustrations

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle as pkl
from functools import reduce
import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Chargement des données (base MovieLens + enrichissements)

In [2]:
# Chargement des données MovieLens

print("Chargement des données MoviesLens")
for fname in glob.glob("ml-20m/*.csv"):
    print("\t {}...".format(fname))
    name = fname.split('/')[1].split('.')[0].replace('-', '_')
    vars()[name] = pd.read_csv(fname, encoding='utf8')
print("Done")

movies.head()
genome_scores.head()
genome_tags.head()
links.head()
ratings.head()
tags.head()

Chargement des données MoviesLens
	 ml-20m/movies.csv...
	 ml-20m/genome-scores.csv...
	 ml-20m/genome-tags.csv...
	 ml-20m/links.csv...
	 ml-20m/ratings.csv...
	 ml-20m/tags.csv...
Done


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [3]:
# Chargement des données complémentaires

print("Chargement des données complémentaires")
for fname in glob.glob("ml-20m/*.pkl"):
    print("\t {}".format(fname))
    name = fname.split('/')[1].split('_')[0]
    vars()[name] = pkl.load(open(fname, "rb"))
print("Done")



Chargement des données complémentaires
	 ml-20m/act_v2.pkl
	 ml-20m/film_v2.pkl
	 ml-20m/crew_v2.pkl
Done


In [4]:
len(tags.tag.unique())
genome_tags.shape

38644

(1128, 2)

In [5]:
act[0][0]

{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/xxPMucou2wRDxLrud8i2D4dsywh.jpg'}

In [6]:
crew[0][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [7]:
film[:1]

[{'adult': False,
  'backdrop_path': '/dji4Fm0gCDVb9DQQMRvAI8YNnTz.jpg',
  'genre_ids': [16, 35, 10751],
  'id': 862,
  'original_language': 'en',
  'original_title': 'Toy Story',
  'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
  'popularity': 22.773,
  'poster_path': '/rhIRbceoE9lR4veEXuwCC2wARtG.jpg',
  'release_date': '1995-10-30',
  'title': 'Toy Story',
  'video': False,
  'vote_average': 7.9,
  'vote_count': 9550}]

## Mise en forme des données

In [8]:
unique_cat = set(reduce(lambda a, b: a + b, movies.apply(lambda x: x.genres.split('|'), axis=1)))

In [9]:
movies[list(unique_cat)] = pd.DataFrame(np.zeros((movies.shape[0], len(unique_cat))))
    
for i, row in movies.iterrows():
    movies.loc[i, row['genres'].split('|')] = 1

In [10]:
# Calculate ratings mean of users
df_ratings = ratings[['movieId', 'rating']].groupby(['movieId'], as_index=False).mean()

# Count number of ratings by movie
df_ratings['nb_ratings'] = ratings[['userId', 'movieId']].groupby(['movieId']).count()

df_tags = tags[['movieId', 'userId']].groupby(['movieId'], as_index=False).count().rename(
    columns={'userId':'nb_tags'})

df_tags['all_tags'] = tags.groupby(['movieId'], as_index=False)['tag'].apply(list)
df_tags['existing_tags'] = df_tags[['all_tags']].apply(lambda line: list(filter(
    lambda tag: not genome_tags[genome_tags['tag'] == tag].empty, line['all_tags'])), axis=1)

In [11]:
df_tags['existing_tagsId'] = df_tags[['existing_tags']].apply(lambda line: list(map(
    lambda tag: genome_tags[genome_tags['tag'] == tag]['tagId'].iloc[0], 
    line['existing_tags'])), axis=1)

In [12]:
df_tags['existing_tagsRelevance'] = df_tags[['movieId', 'existing_tagsId']].apply(
    lambda line: list(map(lambda tagId: 
                          genome_scores[(genome_scores['movieId'] == line.movieId) & \
                                        (genome_scores['tagId'] == tagId)]['relevance'], 
                          line['existing_tagsId'])), axis=1)

In [19]:
df_tags.head()

,movieId,nb_tags,all_tags,existing_tags,existing_tagsId,existing_tagsRelevance
0,1,436,"[Watched, computer animation, Disney animated ...","[computer animation, animation, family, witty,...","[244, 64, 374, 1114, 29, 63, 64, 220, 230, 244...","[[0.9985], [0.98575], [0.8485], [0.77675], [0...."
1,2,123,"[time travel, adapted from:book, board game, c...","[time travel, adapted from:book, scary, time, ...","[1028, 22, 882, 1026, 1028, 377, 882, 1028, 58...","[[0.439], [0.602], [0.685], [0.4495], [0.439],..."
2,3,18,"[old people that is actually funny, sequel fev...","[sequel, sequel, comedy]","[901, 901, 230]","[[0.9505], [0.9505], [0.8467499999999999]]"
3,4,6,"[chick flick, revenge, characters, chick flick...","[chick flick, revenge, chick flick]","[201, 850, 201]","[[0.8665], [0.243], [0.8665]]"
4,5,22,"[Diane Keaton, family, sequel, Steve Martin, w...","[family, sequel, wedding, pregnancy, remake, f...","[374, 901, 1102, 808, 849, 374, 808, 1102]","[[0.8227500000000001], [0.937], [0.80675000000..."


In [79]:
df_tags['mean_relevance'] = df_tags[['existing_tagsRelevance']].iloc[:5, :].apply(
    lambda line sum , axis=1)

In [86]:
df_tags.to_csv("Register.csv", index=False)

In [83]:
df_tags.head()

,movieId,nb_tags,all_tags,existing_tags,existing_tagsId,existing_tagsRelevance,mean_relevance
0,1,436,"[Watched, computer animation, Disney animated ...","[computer animation, animation, family, witty,...","[244, 64, 374, 1114, 29, 63, 64, 220, 230, 244...","[[0.9985], [0.98575], [0.8485], [0.77675], [0....","[[[0.9985], [0.98575], [0.8485], [0.77675], [0..."
1,2,123,"[time travel, adapted from:book, board game, c...","[time travel, adapted from:book, scary, time, ...","[1028, 22, 882, 1026, 1028, 377, 882, 1028, 58...","[[0.439], [0.602], [0.685], [0.4495], [0.439],...","[[[0.439], [0.602], [0.685], [0.4495], [0.439]..."
2,3,18,"[old people that is actually funny, sequel fev...","[sequel, sequel, comedy]","[901, 901, 230]","[[0.9505], [0.9505], [0.8467499999999999]]","[[[0.9505], [0.9505], [0.8467499999999999]]]"
3,4,6,"[chick flick, revenge, characters, chick flick...","[chick flick, revenge, chick flick]","[201, 850, 201]","[[0.8665], [0.243], [0.8665]]","[[[0.8665], [0.243], [0.8665]]]"
4,5,22,"[Diane Keaton, family, sequel, Steve Martin, w...","[family, sequel, wedding, pregnancy, remake, f...","[374, 901, 1102, 808, 849, 374, 808, 1102]","[[0.8227500000000001], [0.937], [0.80675000000...","[[[0.8227500000000001], [0.937], [0.8067500000..."


In [89]:
test = pd.read_csv('Register.csv')

In [93]:
test['existing_tagsRelevance']

0        [243    0.9985\nName: relevance, dtype: float6...
1        [2155    0.439\nName: relevance, dtype: float6...
2        [3156    0.9505\nName: relevance, dtype: float...
3        [3584    0.8665\nName: relevance, dtype: float...
4        [4885    0.82275\nName: relevance, dtype: floa...
5        [6480    0.89175\nName: relevance, dtype: floa...
6        [7630    0.9195\nName: relevance, dtype: float...
7        [7917    0.78525\nName: relevance, dtype: floa...
8        [9042    0.9745\nName: relevance, dtype: float64]
9        [10153    0.99975\nName: relevance, dtype: flo...
10       [12090    0.9975\nName: relevance, dtype: floa...
11       [12867    0.34775\nName: relevance, dtype: flo...
12                                                      []
13       [14798    0.95625\nName: relevance, dtype: flo...
14       [16575    0.99025\nName: relevance, dtype: flo...
15       [18004    0.9795\nName: relevance, dtype: floa...
16       [18069    0.973\nName: relevance, dtype: float.

## Feature engineering
A vous de créer les caractéristiques de description des données qui permettront d'améliorer les performances dans les tâches que vous aurez choisi d'aborder dans le projet.

In [ ]:
# Faire un dictionnaire avec tous les acteurs (acteur => indice)
# + un dictionnaire inversé (indice => acteur)
actors = dict()
actors_inv = dict()
for lista in act:
    for a in lista:
        # affecte une valeur à une clé si la clé n'est pas utilisée
        res = actors.setdefault(a['name'], len(actors))
        if res == len(actors)-1:
            actors_inv[len(actors)-1] = a['name']

# Exemple de transformation supplémentaire
# Dans combien de films de base joue Tom Hanks? (Réponse 57)
# Dans combien de comédies...

# => On voit qu'il est possible de créer facilement des nouvelles caractéristiques qui
# apporteront des informations utiles pour certaines tâches
